In [ ]:

import pandas as pd
import tqdm.auto as tqdm
import numpy as np

# Set device to GPU if available


def LocationCalculate(x, y, z, w):
    X = 2 * x * z + 2 * y * w
    Y = 2 * y * z - 2 * x * w
    Z = 1 - 2 * x * x - 2 * y * y
    a = np.arccos(np.sqrt(X ** 2 + Z ** 2) / np.sqrt(X ** 2 + Y ** 2 + Z ** 2))
    ver = np.where(Y > 0, a / np.pi * 180, -a / np.pi * 180)
    b = np.arccos(X / np.sqrt(X ** 2 + Z ** 2))
    hor = np.where(Z < 0, b / np.pi * 180, (2. - b / np.pi) * 180)
    return (90 - ver) / 180, hor / 360

    return (90 - ver) / 180, hor / 360
def read_csv_file(user_id, video_id):
    filepath = f'Experiment_1/{user_id}/video_{video_id}.csv'
    df = pd.read_csv(filepath,skiprows=[1,2,3,4,5,6,7,8,9,10])
    df['video_id'] = video_id  # 添加video_id字段
    return df

data_with_labels = []

for user_id in range(1, 49):
    user_data = []
    for video_id in range(0, 1):
        df = read_csv_file(user_id, video_id)
        x=df['UnitQuaternion.x']
        y=df['UnitQuaternion.y']
        z=df['UnitQuaternion.z']
        w=df['UnitQuaternion.w']
        Lx,Ly=LocationCalculate(x,y,z,w)
        df['Locationx']=Lx
        df['Locationy']=Ly
        df['classedx1']="0"
        df['classedy1']="0"
        df['classedx2']="0"
        df['classedy2']="0"
        # 添加标签（用户ID）
        df['user_id'] = user_id

        user_data.append(df)

    user_data_combined = pd.concat(user_data, ignore_index=True)
    data_with_labels.append(user_data_combined)

data_with_labels = pd.concat(data_with_labels, ignore_index=True)


In [ ]:
import os
import cv2
import pandas as pd
from tqdm import tqdm
import torch
import numpy as np
from ultralytics import YOLO
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

yolo_model = YOLO('yolov8n.pt')

def extract_frame_features(frame, yolo_model, timestamp):
    features = []
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = yolo_model(image)  # Perform object detection
    #print(results[0].boxes)
    #class_id = results[0].boxes.cls.cpu().numpy()
    obj = results[0].boxes.xyxy.cpu().numpy()
    #判断obj是否为空数组
    if obj.size>0:

        x1=obj[0][0]
        y1=obj[0][1]
        x2=obj[0][2]
        y2=obj[0][3]
    else:
        x1=0
        y1=0
        x2=0
        y2=0

    features.append([x1,y1,x2,y2])  # Process results into a feature vector
    return features


def capture_frame(video_path, target_time):
    # 加载视频文件
    video = cv2.VideoCapture(video_path)
    # 获取视频的帧率
    fps = int(video.get(cv2.CAP_PROP_FPS))
    # 计算目标帧的索引
    target_frame_index = int(target_time * fps)
    # 设置视频文件的当前帧索引
    video.set(cv2.CAP_PROP_POS_FRAMES, target_frame_index)
    # 读取目标帧
    ret, frame = video.read()
    if ret:
        # 保存目标帧为图片
        result=extract_frame_features(frame, yolo_model, target_time)
        return (result)
    # 释放视频资源
    video.release()
for i in tqdm(range(len(data_with_labels))):
    result=capture_frame(f'Experiment_1/1-1-Conan Gore Fly.mp4', data_with_labels['PlaybackTime'][i])
    x1=result[0][0]
    y1=result[0][1]
    x2=result[0][2]
    y2=result[0][3]
    data_with_labels['classedx1'][i]=x1*data_with_labels['Locationx'][i]
    data_with_labels['classedy1'][i]=y1*data_with_labels['Locationy'][i]
    data_with_labels['classedx2'][i]=x2*data_with_labels['Locationx'][i]
    data_with_labels['classedy2'][i]=y2*data_with_labels['Locationy'][i]



In [1]:
import pandas as pd
data_with_labels=pd.read_csv('merged_data.csv')

In [ ]:
from datetime import timedelta
import numpy as np
from tqdm.auto import tqdm

def process_data(data):
    data['Timestamp'] = pd.to_datetime(data['Timestamp'])
    data['rounded_time'] = data['Timestamp'].dt.floor('100ms')

    train_data = []
    train_labels = []
    test_data = []
    test_labels = []

    for user_id, user_data in tqdm(data.groupby('user_id')):
        for video_id, video_data in user_data.groupby('video_id'):
            for second, second_data in video_data.groupby(video_data['rounded_time'].dt.floor('1s')):
                time_slices = []
                for _, chunk_data in second_data.groupby(pd.cut(second_data['Timestamp'], pd.date_range(second, second + timedelta(seconds=1), periods=11))):
                    if not chunk_data.empty:
                        chunk_data = chunk_data[['UnitQuaternion.x', 'UnitQuaternion.y', 'UnitQuaternion.z', 'UnitQuaternion.w', 'HmdPosition.x', 'HmdPosition.y', 'HmdPosition.z', 'objectx', 'objecty']]
                        time_slices.append(chunk_data.mean().values)
                if len(time_slices) == 10:
                    if video_id == 'video_0':
                        test_data.append(time_slices)
                        test_labels.append(user_id)
                    else:
                        train_data.append(time_slices)
                        train_labels.append(user_id)

    return np.array(train_data), np.array(train_labels), np.array(test_data), np.array(test_labels)

train_data, train_labels, test_data, test_labels = process_data(data_with_labels)



  0%|          | 0/48 [00:00<?, ?it/s]

In [5]:
import numpy as np
print(np.__version__)

1.19.5


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Reshape,Bidirectional
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
# 将用户ID转换为从0开始的整数
unique_labels = np.unique(np.concatenate([train_labels, test_labels]))
label_map = {label: idx for idx, label in enumerate(unique_labels)}
int_train_labels = np.array([label_map[label] for label in train_labels])
int_test_labels = np.array([label_map[label] for label in test_labels])

# 将标签数据转换为one-hot编码
one_hot_train_labels = to_categorical(int_train_labels, num_classes=len(unique_labels))
one_hot_test_labels = to_categorical(int_test_labels, num_classes=len(unique_labels))
import tensorflow as tf
from tensorflow.keras.layers import Layer

class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1]), initializer='random_normal', trainable=True)
        self.b = self.add_weight(shape=(input_shape[-1],), initializer='zeros', trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        q = tf.nn.tanh(tf.linalg.matmul(x, self.W) + self.b)
        a = tf.nn.softmax(tf.reduce_sum(q * x, axis=-1), axis=-1)
        return tf.reduce_sum(a[:, :, tf.newaxis] * x, axis=1)

with tf.device('/device:GPU:0'):
    # 定义模型
    model = Sequential()
    model.add(Bidirectional(LSTM(64, input_shape=(10, 9), return_sequences=True)))
#     model.add(AttentionLayer())
#     model.add(Reshape((1, -1)))
    model.add(Bidirectional(LSTM(32)))
    model.add(Dense(len(unique_labels), activation='softmax'))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    # 重塑训练和测试数据
    train_data = train_data.reshape(train_data.shape[0], 10, 9)
    test_data = test_data.reshape(test_data.shape[0], 10, 9)

    # 训练模型
    batch_size = min(128, train_data.shape[0])
    model.fit(train_data, one_hot_train_labels, epochs=150, batch_size=batch_size, validation_data=(test_data, one_hot_test_labels))


Epoch 1/150
868/868 [==============================] - 8s 6ms/step - loss: 3.8413 - accuracy: 0.0329
Epoch 2/150
868/868 [==============================] - 5s 6ms/step - loss: 3.8175 - accuracy: 0.0393
Epoch 3/150
868/868 [==============================] - 5s 6ms/step - loss: 3.8020 - accuracy: 0.0431
Epoch 4/150
868/868 [==============================] - 5s 6ms/step - loss: 3.7847 - accuracy: 0.0463
Epoch 5/150
868/868 [==============================] - 5s 6ms/step - loss: 3.7124 - accuracy: 0.0561
Epoch 6/150
868/868 [==============================] - 5s 6ms/step - loss: 2.9849 - accuracy: 0.1326
Epoch 7/150
868/868 [==============================] - 5s 6ms/step - loss: 2.5956 - accuracy: 0.2016
Epoch 8/150
868/868 [==============================] - 6s 6ms/step - loss: 2.2698 - accuracy: 0.2720
Epoch 9/150
868/868 [==============================] - 6s 7ms/step - loss: 2.0831 - accuracy: 0.3175
Epoch 10/150
868/868 [==============================] - 6s 7ms/step - loss: 1.9377 - accura